# Representing Markov Decision Processes with ProbLog

This notebook aims at discussing how the probabilistic logic programming framework of **ProbLog** can be used to represent **discrete, fully observable, stationary MDPs**.

We illustrate the overall approach by incrementally adding to the language the basic components that are necessary to precisely define Markovian transition functions and deterministic reward functions.

For simplicity of presentation, we use as a running example a well-known domain from the probabilistic planning community (e.g., SysAdmin).

We assume prior knowledge of the ProbLog language.
If you're not familiar with the basics of the ProbLog API, please refer to *Introduction to Probability Theory Using ProbLog* tutorial. 

## 0. Imports & Helper Functions

In [1]:
from problog.program import PrologString
from problog.logic   import Term, Constant
from problog.engine  import DefaultEngine
from problog import get_evaluatable

import itertools

engine = DefaultEngine()

def precompile(model_str):
    db = engine.prepare(PrologString(model_str))
    return db

def extend(db, model_str):
    db2 = db.extend()
    for statement in PrologString(model_str):
        db2 += statement
    return db2

def evaluate(db):
    return get_evaluatable().create_from(db).evaluate()

def query(model, queries, evidence=None):
    if isinstance(model, str):
        db = precompile(model)
    else:
        db = model
    if evidence is None:
        evidence = []
    lf = engine.ground_all(db, queries=queries, evidence=evidence)
    return evaluate(lf)

def run(model_str):
    db = precompile(model_str)
    return evaluate(db)


# imports
builtins = '''
:- use_module(library(lists)).
:- use_module(library(apply)).
'''

# database initialization
db = precompile(builtins)


# SysAdmin constant terms
TMAX = 10
timestep = Term('timestep')
computer = Term('computer')
c1 = Term('c1')
c2 = Term('c2')
c3 = Term('c3')
t0 = Constant(0)
t1 = Constant(1)
computers = [c1, c2, c3]
running = Term('running')
reboot = Term('reboot')


## 1. Timesteps, Fluents and State Space

First things first! To model a SysAdmin problem we need to specify the computers and the network topology. We can do this in ProbLog simply defining a bunch of logical facts. Built-in Prolog lists can come in handy now!

In [2]:
topology = '''
network([c1, c2, c3]).
computer(c1). computer(c2). computer(c3).
connected(c1, [c2, c3]).
connected(c2, [c1]).
connected(c3, [c1]).
'''
db = extend(db, topology)

In order to represent the state of the network in a given timestep, we shall define a set of fluents (i.e., a temporal predicates) that specifies if each $computer$ is $running$ properly at time $T$.

Also, we need to enconde the decision timesteps of the MDP. We do this by defining a predicates $timestep$ and $horizon$.

In the example below we define random states for a finite horizon size $TMAX$.

In [3]:
steps = '''
horizon({0}).
timestep(0).
timestep(T) :- horizon(TMAX), T < TMAX, T2 is T-1, timestep(T2).
'''.format(TMAX)

db = extend(db, steps)
timesteps = []
for i in range(TMAX):
    timesteps.append(timestep(Constant(i)))
print('Timesteps:')
print(list(query(db, queries=timesteps)))
print()

fluents = '''
0.5::running(T, C) :- timestep(T), computer(C).
'''
db2 = extend(db, fluents)
print("Fluents:")
for i in range(TMAX):
    print(query(db2, queries=[running(Constant(i), None)]))
print()

Timesteps:
[timestep(0), timestep(1), timestep(2), timestep(3), timestep(4), timestep(5), timestep(6), timestep(7), timestep(8), timestep(9)]

Fluents:
{running(0,c1): 0.5, running(0,c2): 0.5, running(0,c3): 0.5}
{running(1,c1): 0.5, running(1,c2): 0.5, running(1,c3): 0.5}
{running(2,c1): 0.5, running(2,c2): 0.5, running(2,c3): 0.5}
{running(3,c1): 0.5, running(3,c2): 0.5, running(3,c3): 0.5}
{running(4,c1): 0.5, running(4,c2): 0.5, running(4,c3): 0.5}
{running(5,c1): 0.5, running(5,c2): 0.5, running(5,c3): 0.5}
{running(6,c1): 0.5, running(6,c2): 0.5, running(6,c3): 0.5}
{running(7,c1): 0.5, running(7,c2): 0.5, running(7,c3): 0.5}
{running(8,c1): 0.5, running(8,c2): 0.5, running(8,c3): 0.5}
{running(9,c1): 0.5, running(9,c2): 0.5, running(9,c3): 0.5}



## 2. Transition Model

Before diving into the action space and the MDP transition model definitions, it is easier to start off by defining a simple Markov Chain that models the exogenous effects induced by the interaction of the computers connected together in the network.

### 2.1 State transition

We define the Markov chain by specifying a prior distribution over the state variables at the initial $timestep$ and a conditional distribution for the state transition. We shall do this in a factored form as shown below.

Note that for this example, we've chosen a uniform prior distribution.

In [4]:
prior = '''
0.5::running(0, C) :- computer(C).
'''
db3 = extend(db, prior)

transition = '''
0.05::running(Tnext, C) :- Tnext > 0, computer(C), T is Tnext - 1, not running(T, C).
P::running(Tnext, C)    :- Tnext > 0, computer(C), T is Tnext - 1, running(T, C),
                           connected(C, Ls), length(Ls, TC),
                           include(running(T), Ls, Alive), length(Alive, TR),
                           P is 0.45+0.50*TR/TC.
'''
db3 = extend(db3, transition)

t1 = Constant(1)
queries = [running(t1, c) for c in computers]

t0 = Constant(0)
for values in itertools.product(*[(True, False)]*3):
    evidence = []
    for c, val in zip(computers, values):
        evidence.append((running(t0, c), val))
    print("Initial state:", evidence)
    print("Transition:", query(db3, queries, evidence))
    print()


Initial state: [(running(0,c1), True), (running(0,c2), True), (running(0,c3), True)]
Transition: {running(1,c1): 0.95, running(1,c2): 0.95, running(1,c3): 0.95}

Initial state: [(running(0,c1), True), (running(0,c2), True), (running(0,c3), False)]
Transition: {running(1,c1): 0.6999999999999998, running(1,c2): 0.95, running(1,c3): 0.05000000000000001}

Initial state: [(running(0,c1), True), (running(0,c2), False), (running(0,c3), True)]
Transition: {running(1,c1): 0.6999999999999998, running(1,c2): 0.05000000000000001, running(1,c3): 0.95}

Initial state: [(running(0,c1), True), (running(0,c2), False), (running(0,c3), False)]
Transition: {running(1,c1): 0.45000000000000007, running(1,c2): 0.05000000000000003, running(1,c3): 0.05000000000000003}

Initial state: [(running(0,c1), False), (running(0,c2), True), (running(0,c3), True)]
Transition: {running(1,c1): 0.05000000000000001, running(1,c2): 0.45, running(1,c3): 0.45}

Initial state: [(running(0,c1), False), (running(0,c2), True), (run

### 2.2 Action space

Right now we are almost in condition to specify the full MDP transition model. But first, we shall represent the action space of the SysAdmin domain.

For this particular example, we won't allow any sort of action concurrency, meaning that no two computers can be rebooted at the same time. In order to capture this constraint, we have a number modeling options in ProbLog.

If we know in advance the total number of computers in the network and we are satisfied with a more rigid model we can directly use annotated disjunctions to represent a multinomial distribution over the set of actions. Since in an MDP setting we almost always consider cases where the model is conditioned on a given action, any multinomial could be used. We stick to the uniform distribution for simplicity.

In [5]:
actions = '''
1/4::reboot(c1);1/4::reboot(c2);1/4::reboot(c3);1/4::reboot(none).
'''

print(query(actions, queries=[reboot(None)]))

{reboot(c1): 0.25, reboot(c2): 0.25, reboot(c3): 0.25, reboot(none): 0.25}


If we do not know the number of computers in advance we can build a more flexible and still capture the non-concurrency property by defining constraints with the help of $evidence$.

In [6]:
actions = '''
0.5::reboot(C) :- computer(C).
0.5::reboot(none).

concurrency :- reboot(C1), reboot(C2), C1 \== C2.
reboot :- reboot(X).
'''
db4 = extend(db, actions)
print(query(db4, queries=[reboot(None)]))

evidence = [(Term('concurrency'), False), (reboot(), True)]
print(query(db4, queries=[reboot(None)], evidence=evidence))

{reboot(c1): 0.5, reboot(c2): 0.5, reboot(c3): 0.5, reboot(none): 0.5}
{reboot(c1): 0.25, reboot(c2): 0.25, reboot(c3): 0.25, reboot(none): 0.25}


### 2.3 State-action transition

With a convenient way to specify the action space, we can proceed to define the full state-action transition of the MDP for the SysAdmin problem.

In [7]:
transition = '''
0.5::running(0, C) :- computer(C).

0.5::reboot(C) :- computer(C).
0.5::reboot(none).

concurrency :- reboot(C1), reboot(C2), C1 \== C2.
reboot :- reboot(X).

running(Tnext, C) :- Tnext > 0, computer(C), reboot(C).
0.05::running(Tnext, C) :- Tnext > 0, computer(C), T is Tnext - 1, not reboot(C), not running(T, C).
P::running(Tnext, C)    :- Tnext > 0, computer(C), T is Tnext - 1, not reboot(C), running(T, C),
                           connected(C, Ls), length(Ls, TC),
                           include(running(T), Ls, Alive), length(Alive, TR),
                           P is 0.45+0.50*TR/TC.
'''
db5 = extend(db, transition)
evidence = [(Term('concurrency'), False), (reboot(), True)]
print(query(db5, queries=[running(t1,None)], evidence=evidence))

evidence += [(running(t0,c1), True), (running(t0,c2), True), (running(t0,c3), False)]
evidence += [(reboot(c3), True)]
print(query(db5, queries=[running(t1,None)], evidence=evidence))


{running(1,c1): 0.5312499999999999, running(1,c2): 0.5312499999999999, running(1,c3): 0.5312499999999999}
{running(1,c1): 0.7, running(1,c2): 0.95, running(1,c3): 1.0}
